In [1]:
using JLD,JLD2,Knet
using Base.Iterators: flatten

In [2]:
include("types.jl")
include("pre_processing.jl")
include("encoder_utils.jl")

fillvecs! (generic function with 1 method)

In [3]:
d = JLD.load("pretrained_model.jld2")
v = create_vocab(d)
corpus = load_conllu("tr_imst-ud-train.conllu",v)
wmodel = makewmodel(d)
fillvecs!(wmodel, corpus, v)

┌ Warning: `getval` is deprecated, use `value` instead.
│   caller = #lmloss#18(::Array{Float64,1}, ::Function, ::Array{Array{Float32,2},1}, ::Array{Array{Int64,1},1}, ::Array{Array{Float32,1},1}, ::Array{Any,1}, ::Array{Any,1}) at encoder_utils.jl:59
└ @ Main /home/ilayda/GBDP/encoder_utils.jl:59


In [4]:
corpus

3685-element Array{Any,1}:
 Sanal parçacıklar sa bunların hiçbirini yapamazlar .                                                                                                                                                                                                                   
 Ona her şeyimi verdim .                                                                                                                                                                                                                                                
 Karşısında , pantolonu dizlerine dek ıslak , önlük torbası ham eriklerle dolu İbrahim dikiliyordu .                                                                                                                                                                    
 Nereden biliyorsunuz .                                                                                                                                                           

In [5]:
s = corpus[1]

Sanal parçacıklar sa bunların hiçbirini yapamazlar . 

In [6]:
ev = extend_vocab!(v, corpus)

ExtendedVocab(Dict("Abr"=>26,"PCNom"=>18,"Dup"=>27,"PCIns"=>30,"Since"=>32,"Postp"=>31,"Interj"=>24,"Ques"=>14,"Demons"=>5,"NAdj"=>11…), Dict("VerbForm=Vnoun"=>43,"Person=3"=>4,"Case=Acc"=>11,"Number=Sing"=>7,"Mood=Nec"=>54,"Aspect=Prog"=>27,"Person[psor]=3"=>13,"VerbForm=Conv"=>44,"Aspect=Hab"=>15,"Voice=CauPass"=>46…), Vocab(Dict('\x00\x00\x00\x42'=>37,'\x00\x00\x20\x1d'=>116,'\x00\x00\x00\x58'=>87,'\x00\x00\x20\x44'=>110,'\x00\x00\x00\x56'=>59,'\x00\x00\x00\xd7'=>107,'\x00\x00\x00\x48'=>29,'\x00\x00\x00\x75'=>5,'\x00\x00\x00\x50'=>11,'\x00\x00\x00\x7a'=>61…), Dict("çıkaramayacaklarını"=>7805,"apartmanın"=>13565,"Güreş"=>2733,"karıştı"=>1790,"ikibin'de"=>7225,"formülü"=>4676,"ANAP'ı"=>4506,"ihmal"=>5116,"Virgülden"=>9483,"il"=>6725…), Dict("null"=>8668,"Secure"=>7932,"Puppy"=>9040,"progression"=>11757,"gathered"=>6954,"Core"=>3600,"lovers"=>8832,"underground"=>9093,"backup"=>5047,"caught"=>2501…), "<s>", "</s>", "<unk>", '\x00\x00\x00\x12', '\x00\x00\x00\x13', '\x00\x00\x00\x11', Dic

In [7]:
createfeatvec!(corpus)

In [8]:
padfeatvec!(corpus)

In [9]:
function mlp(w,x)
    for i=1:2:length(w)-2
        x = relu.(w[i]*x .+ w[i+1])
    end
    return w[end-1]*x .+ w[end]
end

mlp (generic function with 1 method)

In [10]:
function create_input_matrix(corpus)
    input = []
    for i in 1:length(corpus)
        for j in 1:length(corpus[i].word)
            push!(input,corpus[i].cavec[j])
        end
    end
    return transpose(hcat(input...))
end

create_input_matrix (generic function with 1 method)

In [11]:
input = create_input_matrix(corpus)

37918×965 LinearAlgebra.Transpose{Real,Array{Real,2}}:
 -1.26117e-44  -0.00206863   -0.348087   …   0   0   0   0   0  0  0  0  0  0
 -0.0          -0.000105809  -0.999725       0   0   0   0   0  0  0  0  0  0
 -1.43166e-26  -0.96306      -0.891741       4   8   0   0   0  0  0  0  0  0
 -0.0          -0.0692767     0.126127      10   0   0   0   0  0  0  0  0  0
 -0.0          -0.000671956  -0.999997       4  13  14   0   0  0  0  0  0  0
 -0.0          -0.0170195    -0.99987    …   4  17   8   0   0  0  0  0  0  0
 -4.85072e-15   0.631611      0.73056        0   0   0   0   0  0  0  0  0  0
 -2.69957e-40  -0.994933     -0.779651      19   0   0   0   0  0  0  0  0  0
 -2.69957e-40  -0.994933     -0.779651       0   0   0   0   0  0  0  0  0  0
 -0.0          -0.00137349   -0.944697       4  20   0   0   0  0  0  0  0  0
 -0.0          -0.00137349   -0.944697   …  22  23  24   0   0  0  0  0  0  0
 -4.85072e-15   0.631611      0.73056        0   0   0   0   0  0  0  0  0  0
 -0.0    

In [12]:
function findmaxnumofwords(corpus)
    max_ = 0
    for i in 1:length(corpus)
        if length(corpus[i].word) > max_
            max_ = length(corpus[i].word)
        end
    end
    return max_
end

findmaxnumofwords (generic function with 1 method)

In [13]:
totalnumofwords = size(input,1)
totalnumoffeatures = size(input,2)
#max number of words that can be head in a sentence, number of columns for head output
numofhead = findmaxnumofwords(corpus)

numofdeprel = length(keys(UDEPREL))

37

In [14]:
totalnumofwords

37918

In [15]:
struct Linear; w; b; end
Linear(i::Int,o::Int,scale=0.01) = Linear(scale * randn(o,i), zeros(o))
batchsize = totalnumoffeatures
# This one allows instances to be defined using input and output sizes:
model_head = Linear(totalnumoffeatures,numofhead)

Linear([-0.011069 0.020075 … -0.00774068 0.0071282; 0.0119896 0.0233192 … 0.00423888 -0.0169738; … ; 0.00562591 0.0054656 … -0.0115236 -0.0054924; -0.00639511 0.0140415 … -0.00637549 0.0115399], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [16]:
function createoutputheadmatrix(corpus)
    outputhead = []
    for i in 1:length(corpus)
        push!(outputhead, corpus[i].head)
    end
    return vcat(outputhead...)
end

createoutputheadmatrix (generic function with 1 method)

In [17]:
outputhead = createoutputheadmatrix(corpus)

37918-element Array{Int16,1}:
  2
  6
  2
  5
  6
  0
  6
  4
  4
  2
  0
  4
 14
  ⋮
 23
 27
 29
 27
 30
 33
 33
 33
  0
 33
 33
 33

In [18]:
(m::Linear)(x) = m.w * x  .+ m.b

In [19]:
ypred = model_head(transpose(input))

57×37918 Array{Float64,2}:
  0.0487514    0.172209     0.176784   …   0.189082     0.206849  
  0.0297051    0.0786793   -0.171697       0.1964       0.2191    
 -0.0556483   -0.107392    -0.0266461      0.0382266    0.050543  
 -0.0767017   -0.102559    -0.12744       -0.0620159   -0.0648712 
 -0.145578    -0.0964007   -0.225958      -0.213302    -0.202106  
  0.0222121   -0.0861415   -0.100643   …  -0.0222819   -0.0210301 
 -0.0661469   -0.0832804   -0.323503      -0.483773    -0.498799  
 -0.0149115    0.0639541    0.0715107      0.0989472    0.101329  
 -0.177335    -0.192154    -0.189258       0.0138225    0.00958456
  0.0191917   -0.0464829   -0.27806       -0.0291775   -0.0259536 
  0.0858667    0.16703      0.0398327  …  -0.127651    -0.10206   
  0.00556456   0.13023      0.15102       -0.111653    -0.0878171 
  0.0467678    0.144115     0.333738      -0.0494457   -0.00878315
  ⋮                                    ⋱                          
  0.0985541   -0.0276165   -0.07592

In [20]:
accuracy(model_head,transpose(input),outputhead)

0.00925681734268685

In [21]:
function nll(scores, y)
    expscores = exp.(scores)
    probabilities = expscores ./ sum(expscores, dims=1)
    answerprobs = (probabilities[y[i],i] for i in 1:length(y))
    mean(-log.(answerprobs))
end

nll (generic function with 1 method)

In [22]:
(m::Linear)(x, y) = Knet.nll(m(x), y)
model_head(transpose(input),outputhead)

4.139774876550762

In [23]:
v.cdict

Dict{Char,Int64} with 122 entries:
  '\x00\x00\x00\x42' => 37
  '\x00\x00\x20\x1d' => 116
  '\x00\x00\x00\x58' => 87
  '\x00\x00\x20\x44' => 110
  '\x00\x00\x00\x56' => 59
  '\x00\x00\x00\xd7' => 107
  '\x00\x00\x00\x48' => 29
  '\x00\x00\x00\x75' => 5
  '\x00\x00\x00\x50' => 11
  '\x00\x00\x00\x7a' => 61
  '\x00\x00\x00\x63' => 9
  '\x00\x00\x00\x3b' => 93
  '\x00\x00\x00\x7c' => 85
  '\x00\x00\x00\x4a' => 60
  '\x00\x00\x00\x21' => 83
  '\x00\x00\x00\x6c' => 6
  '\x00\x00\x00\x70' => 12
  '\x00\x00\x00\xae' => 80
  '\x00\x00\x00\x27' => 84
  '\x00\x00\x00\x23' => 98
  '\x00\x00\x00\x3d' => 71
  '\x00\x00\xd0\xa5' => 122
  '\x00\x00\x00\x2d' => 62
  '\x00\x00\x00\x79' => 13
  '\x00\x00\x00\x69' => 17
  ⋮                  => ⋮

In [24]:
a ='i'

'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)

In [25]:
a

'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)

In [26]:
'\x00\x00\x00\x69'

LoadError: syntax: invalid character literal

In [27]:
v.sowchar

'\x00\x00\x00\x12': Malformed UTF-8 (category Ma: Malformed, bad data)

In [28]:
v.idict

Dict{String,Int64} with 13808 entries:
  "çıkaramayacaklarını" => 7805
  "apartmanın"          => 13565
  "Güreş"               => 2733
  "karıştı"             => 1790
  "ikibin'de"           => 7225
  "formülü"             => 4676
  "ANAP'ı"              => 4506
  "ihmal"               => 5116
  "Virgülden"           => 9483
  "il"                  => 6725
  "devrimlerle"         => 11077
  "Kıyı"                => 11182
  "azla"                => 8931
  "Duvarlarla"          => 4778
  "tı"                  => 1253
  "bağlamasını"         => 10389
  "geçme"               => 13795
  "geceye"              => 6674
  "böyle"               => 613
  "yasaklamanın"        => 422
  "anlayışıyla"         => 8282
  "yaklaştığını"        => 12469
  "bakınır"             => 9900
  "buruşturdu"          => 9728
  "eritme"              => 8462
  ⋮                     => ⋮